In [1]:
import pandas as pd
import mwclient

from lol_fandom import SITE
from lol_fandom import get_leagues, get_tournaments
from lol_fandom import get_scoreboard_games, get_scoreboard_players
from lol_fandom import from_response

pd.set_option('display.max_columns', None)

In [2]:
class Team:
    K = 20
    def __init__(self, name):
        self.name = name
        self.win = 0
        self.loss = 0
        self.point = 1000

    def update_point(team1, team2, result):
        # result: team1 win 1, team1 loss 0
        assert isinstance(team1, Team)
        assert isinstance(team2, Team)

        team1_wr = team1.get_winrate(team2)
        team2_wr = team2.get_winrate(team1)
        team1._update_point(team1_wr, result)
        team2._update_point(team1_wr, 1 - result)

    def get_winrate(self, opponent):
        assert isinstance(opponent, Team)

        return 1 / (10 ** ((opponent.point - self.point) / 400) + 1)

    def _update_point(self, winrate, result):
        # result: win 1, loss 0
        assert result == 0 or result == 1

        if result == 1:
            self.win += 1
        else:
            self.loss += 1

        self.point = self.point + self.K * (result - winrate)

    def to_dict(self):
        data = {
            'Win': self.win,
            'Loss': self.loss,
            'WinRate': self.win / (self.win + self.loss) if self.win != 0 else 0,
            'Point': self.point
        }

        return data

In [3]:
leagues = get_leagues()
leagues

,League,League Short,Region,Level,IsOfficial
0,2015 All-Star Event,2015 ASE,International,Showmatch,Yes
1,2015 International Wildcard Tournament,2015 IWCT,International,Primary,Yes
2,2016 International Wildcard Qualifier,IWCQ,International,Primary,Yes
3,All-Star,All-Star,International,Showmatch,Yes
4,Arena of Legends,AOL,North America,Secondary,No
...,...,...,...,...,...
120,Vietnam Championship Series,VCS,Vietnam,Primary,Yes
121,Volcano League,VL,Latin America,Secondary,Yes
122,World Championship,WCS,International,Primary,Yes
123,World Cyber Arena,WCA,China,Primary,No


In [4]:
tournaments = get_tournaments('L.League_Short="LCK" and T.Year=2022')
tournaments

,OverviewPage,League,Region,Split,SplitNumber,Year
0,LCK/2022 Season/Spring Playoffs,LoL Champions Korea,Korea,Spring,1,2022
1,LCK/2022 Season/Spring Season,LoL Champions Korea,Korea,Spring,1,2022
2,LCK/2022 Season/Summer Playoffs,LoL Champions Korea,Korea,Summer,2,2022
3,LCK/2022 Season/Summer Season,LoL Champions Korea,Korea,Summer,2,2022


In [5]:
scoreboard_games = get_scoreboard_games(f'T.OverviewPage="{tournaments["OverviewPage"][1]}"')
scoreboard_games = scoreboard_games.sort_values(by='DateTime UTC', ascending=True).reset_index(drop=True)
scoreboard_games

,OverviewPage,Team1,Team2,WinTeam,LossTeam,DateTime UTC,Team1Score,Team2Score,Winner,Gamelength,Gamelength Number,Team1Bans,Team2Bans,Team1Picks,Team2Picks,Team1Players,Team2Players,Team1Dragons,Team2Dragons,Team1Barons,Team2Barons,Team1Towers,Team2Towers,Team1Gold,Team2Gold,Team1Kills,Team2Kills,Team1RiftHeralds,Team2RiftHeralds,Team1Inhibitors,Team2Inhibitors,Patch,GameId,MatchId,RiotGameId,DateTime UTC__precision
0,LCK/2022 Season/Spring Season,DRX,Liiv SANDBOX,Liiv SANDBOX,DRX,2022-01-12 08:27:00,0,1,2,36:35,36.583333,"Diana,Caitlyn,Twisted Fate,LeBlanc,Viktor","Renekton,Lee Sin,Leona,Jayce,Akali","Graves,Viego,Ryze,Aphelios,Sona","Tryndamere,Xin Zhao,Syndra,Jhin,Yuumi","Kingen,Pyosik,Zeka (Kim Geon-woo),Deft,BeryL","Dove,Croco,Clozer,Envyy,Kael (Kim Jin-hong)",1,4,0,2,8,9,63747,67669,5,14,2,0,0,2,12.1,LCK/2022 Season/Spring Season_Week 1_1_1,LCK/2022 Season/Spring Season_Week 1_1,None,0
1,LCK/2022 Season/Spring Season,DRX,Liiv SANDBOX,Liiv SANDBOX,DRX,2022-01-12 09:25:00,0,2,2,34:30,34.500000,"Diana,Caitlyn,Yuumi,Samira,Syndra","Renekton,Lee Sin,Twisted Fate,Viktor,LeBlanc","Graves,Jarvan IV,Ryze,Aphelios,Thresh","Tryndamere,Xin Zhao,Akali,Ezreal,Leona","Kingen,Pyosik,Zeka (Kim Geon-woo),Deft,BeryL","Dove,Croco,Clozer,Envyy,Kael (Kim Jin-hong)",0,4,1,1,3,9,60674,67152,7,15,1,1,0,1,12.1,LCK/2022 Season/Spring Season_Week 1_1_2,LCK/2022 Season/Spring Season_Week 1_1,None,0
2,LCK/2022 Season/Spring Season,T1,Kwangdong Freecs,T1,Kwangdong Freecs,2022-01-12 11:12:00,1,0,1,37:13,37.216667,"Lee Sin,Ryze,Viktor,LeBlanc,Graves","Renekton,Twisted Fate,Vex,Jayce,None","Gragas,Jarvan IV,Zoe,Ezreal,Karma","Gwen,Xin Zhao,Syndra,Caitlyn,Lux","Zeus,Oner,Faker,Gumayusi,Keria","Kiin,Ellim,FATE (Yoo Su-hyeok),Teddy,Hoit",5,0,1,0,7,2,66455,58806,12,7,2,0,1,0,12.1,LCK/2022 Season/Spring Season_Week 1_2_1,LCK/2022 Season/Spring Season_Week 1_2,None,0
3,LCK/2022 Season/Spring Season,Kwangdong Freecs,T1,T1,Kwangdong Freecs,2022-01-12 12:14:00,0,2,2,25:06,25.100000,"Irelia,Twisted Fate,Ezreal,Zoe,Akshan","Renekton,Ryze,Lux,LeBlanc,Jayce","Graves,Lee Sin,Syndra,Jhin,Yuumi","Jax,Nidalee,Viktor,Caitlyn,Karma","Kiin,Ellim,FATE (Yoo Su-hyeok),Teddy,Hoit","Zeus,Oner,Faker,Gumayusi,Keria",0,3,0,1,2,10,38495,51002,4,18,1,1,0,1,12.1,LCK/2022 Season/Spring Season_Week 1_2_2,LCK/2022 Season/Spring Season_Week 1_2,None,0
4,LCK/2022 Season/Spring Season,Nongshim RedForce,Hanwha Life Esports,Nongshim RedForce,Hanwha Life Esports,2022-01-13 08:17:00,1,0,1,35:53,35.883333,"Yuumi,Karma,Caitlyn,LeBlanc,Leona","Twisted Fate,Renekton,Aphelios,Lulu,Akali","Graves,Lee Sin,Azir,Jinx,Nautilus","Tryndamere,Xin Zhao,Syndra,Jhin,Rakan","Canna,Dread (Lee Jin-hyeok),Bdd,Ghost (Jang Yo...","DuDu (Lee Dong-ju),OnFleek,Karis,SamD,Vsta",4,0,1,0,11,1,67791,57396,19,8,2,0,3,0,12.1,LCK/2022 Season/Spring Season_Week 1_3_1,LCK/2022 Season/Spring Season_Week 1_3,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,LCK/2022 Season/Spring Season,Kwangdong Freecs,Liiv SANDBOX,Kwangdong Freecs,Liiv SANDBOX,2022-03-20 06:08:00,1,0,1,31:59,31.983333,"Caitlyn,Hecarim,Ahri,Karma,Lulu","Nautilus,Leona,Rakan,Alistar,Yuumi","Gragas,Xin Zhao,LeBlanc,Zeri,Seraphine","Gnar,Lee Sin,Ryze,Aphelios,Renata Glasc","Kiin,Ellim,FATE (Yoo Su-hyeok),Teddy,Hoit","Dove,Croco,Clozer,Ice (Yoon Sang-hoon),Kael (K...",3,2,1,0,10,5,59595,51994,16,5,0,2,2,0,12.5,LCK/2022 Season/Spring Season_Week 9_9_1,LCK/2022 Season/Spring Season_Week 9_9,None,0
208,LCK/2022 Season/Spring Season,Liiv SANDBOX,Kwangdong Freecs,Kwangdong Freecs,Liiv SANDBOX,2022-03-20 07:06:00,0,2,2,34:55,34.916667,"Twisted Fate,Ryze,LeBlanc,Xin Zhao,Jarvan IV","Zeri,Caitlyn,Hecarim,Volibear,Tryndamere","Gragas,Viego,Ahri,Jinx,Nautilus","Aatrox,Lee Sin,Vex,Aphelios,Leona","Dove,Croco,Clozer,Ice (Yoon Sang-hoon),Kael (K...","Kiin,Ellim,FATE (Yoo Su-hyeok),Teddy,Hoit",3,2,0,1,3,9,55138,65894,8,21,2,0,0,1,12.5,LCK/2022 Season/Spring Season_Week 9_9

In [6]:
team_names = scoreboard_games[['Team1', 'Team2']].apply(pd.unique)
team_names = list(set(list(team_names['Team1']) + list(team_names['Team2'])))
team_names

['T1',
 'Hanwha Life Esports',
 'Kwangdong Freecs',
 'KT Rolster',
 'Liiv SANDBOX',
 'Nongshim RedForce',
 'DRX',
 'Fredit BRION',
 'Gen.G',
 'DWG KIA']

In [7]:
teams = {}
for name in team_names:
    teams[name] = Team(name)

In [8]:
for row in scoreboard_games.itertuples():
    team1, team2 = row.Team1, row.Team2
    result = 1 if row.WinTeam == team1 else 0
    Team.update_point(teams[team1], teams[team2], result)

In [10]:
elo_ratings = pd.DataFrame(
    data=map(lambda x: x.to_dict(), teams.values()),
    index=teams.keys()
)
elo_ratings = elo_ratings.sort_values(by='Point', ascending=False)
elo_ratings

,Win,Loss,WinRate,Point
T1,36,7,0.837209,1335.143689
Gen.G,28,12,0.700000,1180.723903
DWG KIA,27,15,0.642857,1140.525906
DRX,22,19,0.536585,1046.799126
Kwangdong Freecs,20,22,0.476190,995.985807
Fredit BRION,21,23,0.477273,990.965560
KT Rolster,19,24,0.441860,949.813838
Nongshim RedForce,16,29,0.355556,882.870748
Hanwha Life Esports,12,31,0.279070,835.210151
Liiv SANDBOX,11,30,0.268293,833.148975
